In [1]:
import cv2, numpy, time
from imutils.video import WebcamVideoStream, FPS
%matplotlib inline
import matplotlib as plt
import numpy as np
import socket
import collections
from concurrent import futures
import random
import serial

def nothing(x):
    pass
def take_first(l):
    return l[0]
def pipe_test(pipe):
    while True:
        pipe.appendleft(random.randint(0, 7))
        time.sleep(1)
        
def x_2_sec(x):
    return (10.8 - (x*0.01125))*0.25
        
class Empty():
    pass

def pipe_pusher(pipe):
    print("Hello from pipepusher :-)")
    while True:
        bc = pipe.pop()
        #so.sendto("set {}".format(bc).encode(), ("192.168.178.21", 50006))
        sst.goto_box(bc)
        pipe.appendleft(4)
        time.sleep(0.1)
def draw_pipe(pipe):
    im = np.zeros((8000, 200, 3), np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i, p in enumerate(list(pipe)):
        cv2.putText(im,str(p),(4,20*i), font, 1,(0,255,0),1,cv2.LINE_AA)
    cv2.imshow("pipe", im)
    
    
#[color_index, "color_name", [under_hsv], [upper_hsv], "color_mode"]
hsvcolors = [
    [0, "Red", np.array((150, 0, 0)), np.array((200, 255, 255)), "HSV"],
    [0, "Red1", np.array((0, 0, 0)), np.array((1, 255, 255)), "HSV"],
    [1, "Green", np.array((50, 0, 0)), np.array((100, 255, 255)), "HSV"],
    [2, "Blue", np.array((100, 0, 0)), np.array((150, 255, 255)), "HSV"],
    [3, "Yellow", np.array((20, 0, 0)), np.array((30, 255, 255)), "HSV"],
    [4, "White", np.array((255, 255, 255)), np.array((255, 255, 255)), "BGR"],
    [5, "Orange", np.array((9, 0, 0)), np.array((13, 255, 255)), "HSV"],
    [6, "Black", np.array((10, 10, 10)), np.array((55, 55, 55)), "BGR"],
    [7, "LightGray", np.array((140, 140, 140)), np.array((180, 180, 180)), "BGR"],
    [8, "DarkGray", np.array((40, 40, 40)), np.array((130, 130, 130)), "BGR"],
]
under_conveyor = numpy.array([140, 160, 155])
upper_conveyor = numpy.array([255, 255, 255])
DELAY_OFFSET_BEFORE = 1#in seconds, when positive, slide will turn earlier
DELAY_OFFSET_AFTER = 0#in seconds, when positive, slide will stay longer
DELAY_PER_BOX = 0.078125
def find_out_color(img):
    cv2.imshow("Brick BGR", img)
    #print(img.shape)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.imshow("Brick HSV", hsv)
    px_cnt = img.shape[0]*img.shape[1]
    scores = np.array([])
    conv = cv2.inRange(img, under_conveyor, upper_conveyor)
    for color_index, color_name, under_hls, upper_hls, color_mode in hsvcolors:
        ir = (cv2.inRange(hsv, under_hls, upper_hls) if color_mode=="HSV" else cv2.inRange(img, under_hls, upper_hls))
        ir = cv2.bitwise_and(cv2.bitwise_not(conv), ir)
        cv2.imshow(color_name, ir)
        cnz = cv2.countNonZero(ir)
        #print(color_name, round(cnz/px_cnt*100, 2), "%")
        scores = np.append(scores, cnz/px_cnt*100)
    r = hsvcolors[np.argmax(scores)].copy()
    r.append(np.max(scores))
    #print(r)
    return r
def found_brick(color_number, x, y, w, h):
    front_x = x+w
    remain_px = 933-front_x
    remain_sec = remain_px / 142
    cmd = "set {color} {delay}".format(color = color_number, delay = remain_sec - DELAY_OFFSET_SEC)
    print("sending: ", cmd)
    so.sendto(cmd.encode(), ("192.168.178.21", 50006))
    
class serial_slideturner():
    def __init__(self, num_boxes, comport = "COM4"):
        self.num_boxes = num_boxes
        if num_boxes < 1:
            raise ValueError("You must init at least one box")
        self.comport = comport
        self.ser = serial.Serial(self.comport)
    def calc_angle(self, box, verbose=True):#box is from 0 to self.num_boxes-1
        angle = 180/self.num_boxes * (box+0.5)
        return angle
    def goto_box(self, box, really_delay=0, verbose=False, delay=0):
        if box > self.num_boxes or box < 0:
            return False
        if delay:
            self.thread = Thread(target=self.goto_box, args=(box, delay))
            self.thread.start()
            return
        if really_delay:
            time.sleep(really_delay)
        self.box = box
        angle = self.calc_angle(box, verbose=verbose)
        if angle==10:
            angle += 2
        #msg = "set {}\n".format(int(angle)).encode()
        msg = chr(int(angle)).encode()
        ####print(msg)
        self.ser.write(msg)
        #self.ser.close()

In [ ]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('thresh')
cv2.createTrackbar('T','thresh',0,255,nothing)
newest_brick = -1
newest_x = 10000
brick_colors = {-1: hsvcolors[5]+[100], 0: hsvcolors[5]+[100], 1: hsvcolors[5]+[100]}
for i in range(10000000):
    r = False
    while not r:
        r, fr = cap.read()
    cv2.imshow("input", fr)
    fr_orig = fr.copy()
    conv = cv2.inRange(fr, under_conveyor, upper_conveyor)
    cv2.imshow("conv", conv)
    imgray = cv2.cvtColor(fr,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,cv2.getTrackbarPos('T','thresh'),255,cv2.THRESH_BINARY)
    cv2.imshow("thresh", thresh)
    not_conv = cv2.bitwise_not(conv)
    cv2.imshow("not_conv", not_conv)
    im2, contours, hierarchy = cv2.findContours(not_conv,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    real_contours = []
    real_minareas = []
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if x == y == 0:#whole pic as contour
            continue
        elif w < 40 or h < 40:
            continue
        elif (w < 100 or h < 100) and x < 10:
            continue
        elif w > 600 and h > 400:
            continue
        elif sum(cv2.mean(fr[x:x+w, y:y+h])) > 700:
            continue
        mx = x + (w/2)
        my = y + (h/2)
        real_contours.append([x, y, w, h, mx, my])
    real_contours.sort(key=take_first)
    i = 0
    for x, y, w, h, mx, my in real_contours:
        brickindex = newest_brick - i
        if x < (newest_x - 48):
            newest_brick += 1
            brick_colors[newest_brick] = find_out_color(fr_orig[y:y+h, x:x+w])
            print("found new brick!", brick_colors[newest_brick][1])
            #print("Area: ", mar[1][0]*mar[1][1])
            #found_brick(brick_colors[newest_brick][0], x, y, w, h)
        ####brick_colors[brickindex] = find_out_color(fr_orig[y:y+h, x:x+w])
        if x>1 and x+w<620:
            p_fall = x_2_sec(x+(w/2)) + 0.2
            #print(p_fall)
            sec2 = p_fall - DELAY_OFFSET_BEFORE
            #print(brick_colors[brickindex])
            try:
                bc = brick_colors[brickindex][0]
            except:
                bc = 4
            sec1 = p_fall + DELAY_OFFSET_AFTER + (abs(4-bc)*DELAY_PER_BOX)
            i1 = int(round((10-sec1)*10, 0))
            i2 = int(round((10-sec2)*10, 0))
            #print((i2-i1)*0.16103)#length of brick in cm
            #print(i1, i2)
            for inn in range(i1, i2):
                pipe[inn] = brick_colors[brickindex][0]
        i += 1
        cv2.rectangle(fr, (x, y), (x+w, y+h), (0,0,0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(fr,str(brickindex)+" "+str(brick_colors[brickindex][1])+" "+str(round(brick_colors[brickindex][5], 2))+"%",(x,y+64), font, 1,(0,0,0),2,cv2.LINE_AA)
    if real_contours:
        #print(len(contours), "\t", len(real_contours))
        newest_x = real_contours[0][0]
    cv2.line(fr,(newest_x,0),(newest_x,100),(255,0,0),5)
        
    cv2.imshow("output", fr)
    
    draw_pipe(pipe)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cap.release()

found new brick! Yellow
found new brick! Red
found new brick! Black
found new brick! Blue
found new brick! Blue
found new brick! Blue
found new brick! Blue
found new brick! Blue
found new brick! Yellow
found new brick! Green
found new brick! Yellow
found new brick! Yellow
found new brick! Blue
found new brick! Blue
found new brick! Orange
found new brick! Yellow
found new brick! Orange
found new brick! Green
found new brick! Green
found new brick! Blue
found new brick! Yellow
found new brick! Blue
found new brick! Yellow
found new brick! Black
found new brick! LightGray
found new brick! Green
found new brick! Green


In [ ]:
brick_colors

In [ ]:
brickindex

In [ ]:
cap.release()


In [2]:
sst = serial_slideturner(9, comport="COM3")
sst.goto_box(4)
pipe = collections.deque([-1]*100, maxlen=100)
executor = futures.ThreadPoolExecutor(max_workers=10)
n = executor.submit(pipe_pusher, pipe)

Hello from pipepusher :-)


In [ ]:
n.result()

In [19]:
n

<Future at 0x1b8b9e2da58 state=running>

b'Z'
b'Z'
b'Z'

In [17]:
sst.goto_box(4)

b'Z'


In [13]:
sst.ser.close()

In [12]:
sst.ser.read_all()

b''

In [4]:
so = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
so.sendto("num_boxes 9".encode(), ("192.168.178.21", 50006))

11

In [7]:
while 1:
    for b in range(9):
        so.sendto("set {}".format(b).encode(), ("192.168.178.21", 50006))
        time.sleep(2)

KeyboardInterrupt: 

In [5]:
so.sendto("set 3".encode(), ("192.168.178.21", 50006))

5